In [89]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error

from sklearn.base import BaseEstimator, TransformerMixin

%matplotlib inline

sns.set(style="darkgrid")

In [90]:
data = pd.read_csv(r'/home/constantine/constantine/education/python/hse/taxi_train.csv')

In [91]:
data.drop(['dropoff_datetime'], axis=1, inplace=True)
data.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [92]:
# SAMPLING!!!!
data = data.sample(n=2000, random_state=35)

In [93]:
data = data[data.trip_duration < 20000]
data['log_trip_duration'] = np.log(data['trip_duration'] + 1)
data.drop(['trip_duration', 'vendor_id', 'passenger_count'], axis=1, inplace=True)

In [94]:
# sns.histplot(data['log_trip_duration'])

In [95]:
# sns.relplot(data=data, x="pickup_hour", y="trip_duration", hue="pickup_dayofweek")

In [96]:
# fig = plt.figure(figsize=(10,10))
# ax = plt.axes(projection='3d')
# xval = [0,1,2,3,4,5,6]
# yval = range(24)
# xline = []
# yline = []
# zline = []
# for dayofweek in xval:
#     for hour in yval:
#         xline.append(dayofweek)
#         yline.append(hour)
#         zline.append(data[(data.pickup_dayofweek == dayofweek) & (data.pickup_hour == hour)].shape[0])
# interp_func = interp2d(xline, yline, zline, kind='linear')
# x_mesh, y_mesh = np.meshgrid(xline, yline)
# color = []
# color += ['b'] * 24
# color += ['g'] * 24
# color += ['r'] * 24
# color += ['c'] * 24
# color += ['m'] * 24
# color += ['y'] * 24
# color += ['k'] * 24
# color
# ax.bar(left=xline, height=zline, zs=yline, zdir='y', color=color, alpha=0.8)
# ax.plot_trisurf(xline, yline, zline)

In [97]:
# fig = plt.figure(figsize=(100,100))
# x = data.pickup_dayofyear.values
# y_group = data.groupby(by='pickup_dayofyear')
# y = [y_group.get_group(day).shape[0] for day in x]
# sns.barplot(x, y)
# plt.savefig('cock.pdf')

In [98]:
# data_groupby = data.groupby(['pickup_day', 'pickup_month']).count()
# palette = list(sns.color_palette()[:6])
# fig, ax = plt.subplots(figsize=(20,8))
# sns.lineplot(data=data_groupby, x="pickup_day", y="id", hue="pickup_month", palette=palette, ax=ax)

In [99]:
# data_groupby = data.groupby(['pickup_hour', 'pickup_dayofweek']).count()
# palette = list(sns.color_palette()[:7])
# fig, ax = plt.subplots(figsize=(20,8))
# sns.lineplot(data=data_groupby, x="pickup_hour", y="id", hue="pickup_dayofweek", palette=palette, ax=ax)

In [100]:
# data_groupby_agg_dayofweek = data.groupby(by='pickup_dayofweek').mean()
# data_groupby_agg_dayofweek
# sns.scatterplot(x=data_groupby_agg_dayofweek.index, y=data_groupby_agg_dayofweek['log_trip_duration'])

In [101]:
# data_groupby_agg_hour = data.groupby(by='pickup_hour').mean()
# data_groupby_agg_hour
# sns.scatterplot(x=data_groupby_agg_hour.index, y=data_groupby_agg_hour['log_trip_duration'])

In [102]:
# data_groupby_agg_dayofyear = data.groupby(by='pickup_month').mean()
# data_groupby_agg_dayofyear
# sns.scatterplot(x=data_groupby_agg_dayofyear.index, y=data_groupby_agg_dayofyear['log_trip_duration'])

In [103]:
# from sklearn.model_selection import GridSearchCV
# alphas = np.logspace(-2, 3, 20)
# searcher = GridSearchCV(Ridge(), [{"alpha": alphas}], scoring="neg_root_mean_squared_error", cv=10) 
# searcher.fit(param_train_pickup, target_train_pickup)
# best_alpha = searcher.best_params_['alpha']
# best_alpha

In [104]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso

In [105]:
# 0.014187424435005062
# 0.6198457923348892

In [106]:
class PickupTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, this = True):
        self.this = this
        
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        that = self.this
        df_copy = df.copy()
        df_copy = self.parsing_pickup_datetime(df_copy)
        df_copy = self.outstanding_dates(df_copy)
        return df_copy
    
    def outstanding_dates (self, df):
        df['pickup_outstand'] = np.zeros(df.shape[0], dtype=int)
        df.loc[df['pickup_dayofyear'].isin([21, 22, 23, 151]), ['pickup_outstand']] = 1
        df.drop(['pickup_dayofyear'], axis=1, inplace=True)
        return df
    
    def parsing_pickup_datetime (self, df):
        date_time = df['pickup_datetime'].apply(lambda date: datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S'))
        df['pickup_dayofweek'] = date_time.dt.dayofweek
        df['pickup_hour'] = date_time.dt.hour
        df['pickup_month'] = date_time.dt.month
        df['pickup_dayofyear'] = date_time.dt.dayofyear
        df.drop(['pickup_datetime'], axis=1, inplace=True)
        return df

In [107]:
def show_circles_on_map(data, latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (data[latitude_column].mean(), data[longitude_column].mean())
    m = folium.Map(location=location)

    for _, row in data.iterrows():
        folium.Circle(
            radius=100,
            location=(row[latitude_column], row[longitude_column]),
            color=color,
            fill_color=color,
            fill=True
        ).add_to(m)

    return m

In [108]:
class CoordTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, this = True):
        self.this = this
        
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        that = self.this
        df_copy = df.copy()
        df_copy = self.manh_dist(df_copy)
        df_copy = self.best_sectors(df_copy, 'pickup_sector', 20)
        df_copy = self.best_sectors(df_copy, 'dropoff_sector', 20)
        df_copy.drop(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'],
                    axis=1, inplace=True)
        return df_copy
    
    def sector_bounds(self, df, y=None):
        lng_array = np.concatenate([df['pickup_longitude'].to_numpy(), df['dropoff_longitude'].to_numpy()])
        lng_max = np.max(lng_array)
        lng_min = np.min(lng_array)
        lat_array = np.concatenate([df['pickup_latitude'].to_numpy(), df['dropoff_latitude'].to_numpy()])
        lat_max = np.max(lat_array)
        lat_min = np.min(lat_array)
        return (lat_min, lng_min, lat_max, lng_max)
    
    def divisior(self, df, n_divisions):
        lat_min, lng_min, lat_max, lng_max = self.sector_bounds(df)
        df['pickup_sector'] = np.zeros(df.shape[0])
        for i_lat in range(1, n_divisions + 1):
            lat_2 = lat_min + (lat_max - lat_min) * i_lat / n_divisions
            lat_1 = lat_min + (lat_max - lat_min) * (i_lat - 1) / n_divisions
            for i_lng in range( 1, n_divisions + 1):
                lng_2 = lng_min + (lng_max - lng_min) * i_lng / n_divisions
                lng_1 = lng_min + (lng_max - lng_min) * (i_lng - 1) / n_divisions
                df.loc[((df['pickup_latitude'] <= lat_2) 
                    & (df['pickup_latitude'] >= lat_1)
                    & (df['pickup_longitude'] <= lng_2) 
                    & (df['pickup_longitude'] >= lng_1)), ['pickup_sector']] = f'{i_lat}-{i_lng}'
        for i_lat in range(1, n_divisions + 1):
            lat_2 = lat_min + (lat_max - lat_min) * i_lat / n_divisions
            lat_1 = lat_min + (lat_max - lat_min) * (i_lat - 1) / n_divisions
            for i_lng in range( 1, n_divisions + 1):
                lng_2 = lng_min + (lng_max - lng_min) * i_lng / n_divisions
                lng_1 = lng_min + (lng_max - lng_min) * (i_lng - 1) / n_divisions
                df.loc[((df['dropoff_latitude'] <= lat_2) 
                    & (df['dropoff_latitude'] >= lat_1)
                    & (df['dropoff_longitude'] <= lng_2) 
                    & (df['dropoff_longitude'] >= lng_1)), ['dropoff_sector']] = f'{i_lat}-{i_lng}'
        return df
                
    def best_sectors(self, df, param, n_divisions):    
        df = self.divisior(df, n_divisions)
        keys = list(df.groupby(by=[param]).groups.keys())
        miserable_part = 1 / n_divisions
        for key in keys:
            if df.groupby(by=[param]).get_group(key).shape[0] < df.shape[0] * miserable_part:
                df.loc[df[param] == key, [param]] = 'miserable'
        return df
    
    def manh_dist(self, df):
        df['log_distance'] = np.log(self.manh_dist_val(df['pickup_latitude'], 
                        df['pickup_longitude'],
                        df['dropoff_latitude'],
                        df['dropoff_longitude']) + 1)
        return df
    
    def manh_dist_val(self, lat1, lng1, lat2, lng2):
        lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
        r = 6371
        manh_angle = np.arcsin(274/564)
        x = np.abs(lng2 - lng1) * r * np.cos(lat1)
        y = np.abs(lat2 - lat1) * r 
        x_new = x * np.cos(manh_angle) - y * np.sin(manh_angle)
        y_new = x * np.sin(manh_angle) + y * np.cos(manh_angle)
        return np.abs(x_new) + np.abs(y_new)

In [109]:
# data_coord = data_coord.drop(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
#                             'dropoff_latitude'], axis=1, inplace=True)

In [110]:
# train_coord, test_coord = train_test_split(data_coord, test_size=0.3, random_state=666)
# param_train_coord = train_coord.drop(['log_trip_duration'], axis=1, inplace=False)
# target_train_coord = train_coord['log_trip_duration']
# param_test_coord = test_coord.drop(['log_trip_duration'], axis=1, inplace=False)
# target_test_coord = test_coord['log_trip_duration']

In [111]:
# categorical = ['pickup_sector', 'dropoff_sector']
# numerical = ['log_distance']
# column_transformer_sector = ColumnTransformer([('sector', CoordTransformer())])
# column_transformer = ColumnTransformer([
#          ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical),
#          ('num', StandardScaler(), numerical)])
# pipeline = Pipeline(steps=[
#     ('sector', CoordTransformer()),
#     ('ohe_and_scaling', column_transformer),
#     ('regression', Ridge(alpha=1))
# ])

In [112]:
# pipeline.fit(param_train_coord, target_train_coord)
# prediction_coord = pipeline.predict(param_test_coord)
# print(mean_squared_log_error(y_true=target_test_coord, y_pred=prediction_coord))
# print(mean_absolute_error(y_true=target_test_coord, y_pred=prediction_coord))
#0.00564454002647366
# 0.35840173912341494

In [113]:
data_searcher = data.sample(1000)

train_searcher, test_searcher = train_test_split(data_searcher, test_size=0.3, random_state=6)
param_train_searcher = train_searcher.drop(['log_trip_duration'], axis=1, inplace=False)
target_train_searcher = train_searcher['log_trip_duration']
param_test_searcher = test_searcher.drop(['log_trip_duration'], axis=1, inplace=False)
target_test_searcher = test_searcher['log_trip_duration']

categorical = ['store_and_fwd_flag', 'pickup_sector', 'dropoff_sector', 'pickup_dayofweek','pickup_outstand', 'pickup_month']
numerical = ['log_distance', 'pickup_hour']

from sklearn.model_selection import GridSearchCV 
alphas = np.logspace(-2, 3, 20) 
searcher = GridSearchCV(Ridge(), [{"alpha": alphas}], scoring="neg_root_mean_squared_error", cv=10) 

column_transformer = ColumnTransformer([
         ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical),
         ('num', StandardScaler(), numerical)])
pipeline_searcher = Pipeline(steps=[
    ('pickup', PickupTransformer()),
    ('sector', CoordTransformer()),
    ('ohe_and_scaling', column_transformer),
    ('searcher', searcher)
])
pipeline_searcher.fit(param_train_searcher, target_train_searcher)
# prediction_searcher = pipeline_searcher.predict(param_test_searcher)
best_alpha = pipeline_searcher.named_steps.searcher.best_params_['alpha']
best_alpha

7.847599703514606

In [114]:
train, test = train_test_split(data, test_size=0.3, random_state=666)
param_train = train.drop(['log_trip_duration'], axis=1, inplace=False)
target_train = train['log_trip_duration']
param_test = test.drop(['log_trip_duration'], axis=1, inplace=False)
target_test = test['log_trip_duration']

In [115]:
column_transformer = ColumnTransformer([
         ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical),
         ('num', StandardScaler(), numerical)])
pipeline = Pipeline(steps=[
    ('pickup', PickupTransformer()),
    ('sector', CoordTransformer()),
    ('ohe_and_scaling', column_transformer),
    ('regression', Ridge(alpha=best_alpha))
])
pipeline.fit(param_train, target_train)
prediction = pipeline.predict(param_test)

print(np.sqrt(mean_squared_log_error(y_true=target_test, y_pred=prediction)))
print(mean_absolute_error(y_true=target_test, y_pred=prediction))

0.07418116888502618
0.3531794118034308


In [116]:
# 0.005660402180400106
# 0.3591578094159026
data.head()

,id,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,log_trip_duration
1319811,id3312385,2016-03-21 11:22:05,-74.004913,40.740765,-73.987877,40.750237,N,6.156979
1129847,id0769540,2016-04-26 10:38:33,-73.968849,40.761593,-73.982437,40.746639,N,6.853299
1330797,id1794854,2016-06-17 23:07:31,-73.992340,40.747742,-73.950432,40.658451,N,7.828835
1065047,id2964310,2016-05-12 09:05:53,-74.009468,40.711922,-73.991608,40.749901,N,7.593878
1444476,id1296427,2016-03-26 10:30:11,-73.972549,40.762421,-73.956100,40.784561,N,5.820083


In [117]:
alphas = np.logspace(-2, 3, 20) 
searcher = GridSearchCV(Lasso(), [{"alpha": alphas}], scoring="neg_root_mean_squared_error", cv=10) 

column_transformer = ColumnTransformer([
         ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical),
         ('num', StandardScaler(), numerical)])
pipeline_searcher = Pipeline(steps=[
    ('pickup', PickupTransformer()),
    ('sector', CoordTransformer()),
    ('ohe_and_scaling', column_transformer),
    ('searcher', searcher)
])
pipeline_searcher.fit(param_train_searcher, target_train_searcher)
# prediction_searcher = pipeline_searcher.predict(param_test_searcher)
best_alpha = pipeline_searcher.named_steps.searcher.best_params_['alpha']
print(best_alpha)

pipeline = Pipeline(steps=[
    ('pickup', PickupTransformer()),
    ('sector', CoordTransformer()),
    ('ohe_and_scaling', column_transformer),
    ('regression', Lasso(alpha=best_alpha))
])
pipeline.fit(param_train, target_train)
prediction = pipeline.predict(param_test)

print(np.sqrt(mean_squared_log_error(y_true=target_test, y_pred=prediction)))
print(mean_absolute_error(y_true=target_test, y_pred=prediction))

0.01
0.07550457491182379
0.35238984626308406


In [118]:
print (f'train before split: {train.shape[0]}')
train, validation = train_test_split(train, test_size=0.2, random_state=666)
print(f'train:{train.shape[0]} validation:{validation.shape[0]}')
param_train = train.drop(['log_trip_duration'], axis=1, inplace=False)
target_train = train['log_trip_duration']
param_validation = validation.drop(['log_trip_duration'], axis=1, inplace=False)
target_validation = validation['log_trip_duration']

train before split: 1398
train:1118 validation:280


In [119]:
alphas = np.logspace(-2, 3, 20) 
searcher = GridSearchCV(Ridge(), [{"alpha": alphas}], scoring="neg_root_mean_squared_error", cv=10) 

column_transformer = ColumnTransformer([
         ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical),
         ('num', StandardScaler(), numerical)])
pipeline_searcher = Pipeline(steps=[
    ('pickup', PickupTransformer()),
    ('sector', CoordTransformer()),
    ('ohe_and_scaling', column_transformer),
    ('searcher', searcher)
])
pipeline_searcher.fit(param_validation, target_validation)
# prediction_searcher = pipeline_searcher.predict(param_test_searcher)
best_alpha = pipeline_searcher.named_steps.searcher.best_params_['alpha']
print(best_alpha)

pipeline = Pipeline(steps=[
    ('pickup', PickupTransformer()),
    ('sector', CoordTransformer()),
    ('ohe_and_scaling', column_transformer),
    ('regression', Ridge(alpha=best_alpha))
])
pipeline.fit(param_train, target_train)
prediction = pipeline.predict(param_test)

print(np.sqrt(mean_squared_log_error(y_true=target_test, y_pred=prediction)))
print(mean_absolute_error(y_true=target_test, y_pred=prediction))

4.281332398719392
0.07430530007099712
0.35397954196263337


In [120]:
pipeline_666 = Pipeline(steps=[
    ('pickup', PickupTransformer()),
    ('sector', CoordTransformer()),
])
pipeline_666.fit_transform(param_train)
# попробовать коллаб

,id,store_and_fwd_flag,pickup_dayofweek,pickup_hour,pickup_month,pickup_outstand,log_distance,pickup_sector,dropoff_sector
1013762,id0034039,N,0,7,3,0,1.393085,10-10,12-10
437551,id1900565,N,1,19,4,0,1.377780,10-11,miserable
119467,id3707539,N,3,16,3,0,1.019792,14-11,miserable
96389,id2914469,N,5,11,2,0,0.903544,10-10,9-10
1437680,id1345337,N,6,4,1,0,3.034313,10-10,miserable
...,...,...,...,...,...,...,...,...,...
868445,id3629527,N,4,22,6,0,1.258948,8-10,miserable
874277,id2606200,N,2,13,2,0,1.821922,12-12,11-10
256638,id2912566,N,0,21,2,0,0.657024,11-11,miserable
750003,id1130647,N,2,8,2,0,1.568594,10-10,12-11


In [133]:
class PickupTransformer1(BaseEstimator, TransformerMixin):
    
    def __init__(self, this = True):
        self.this = this
        
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        that = self.this
        df_copy = df.copy()
        df_copy = self.parsing_pickup_datetime(df_copy)
        df_copy = self.outstanding_dates(df_copy)
        return df_copy
    
    def outstanding_dates (self, df):
        df['pickup_outstand'] = np.zeros(df.shape[0], dtype=int)
        df.loc[df['pickup_dayofyear'].isin([21, 22, 23, 151]), ['pickup_outstand']] = 1
        df.drop(['pickup_dayofyear'], axis=1, inplace=True)
        return df
    
    def parsing_pickup_datetime (self, df):
        date_time = df['pickup_datetime'].apply(lambda date: datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S'))
        df['pickup_dayofweek'] = date_time.dt.dayofweek
        df['pickup_hour'] = date_time.dt.hour
        df['pickup_month'] = date_time.dt.month
        df['pickup_dayofyear'] = date_time.dt.dayofyear
        df.drop(['pickup_datetime'], axis=1, inplace=True)
        return df

In [134]:
pipeline_6661 = Pipeline(steps=[
    ('pickup', PickupTransformer1())
])
pipeline_6661.fit_transform(param_train, target_train)
# попробовать коллаб

,id,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_dayofweek,pickup_hour,pickup_month,pickup_outstand
1013762,id0034039,-73.997253,40.744839,-73.981277,40.768314,N,0,7,3,0
437551,id1900565,-73.977844,40.750534,-74.001656,40.745010,N,1,19,4,0
119467,id3707539,-73.969742,40.799961,-73.980011,40.786129,N,3,16,3,0
96389,id2914469,-73.985558,40.741360,-73.992867,40.730068,N,5,11,2,0
1437680,id1345337,-73.987999,40.749535,-73.984467,40.617916,N,6,4,1,0
...,...,...,...,...,...,...,...,...,...,...
868445,id3629527,-73.992241,40.715134,-73.976021,40.730484,N,4,22,6,0
874277,id2606200,-73.953171,40.767826,-73.993973,40.755989,N,2,13,2,0
256638,id2912566,-73.966949,40.757000,-73.972107,40.749741,N,0,21,2,0
750003,id1130647,-73.986641,40.742889,-73.962578,40.767052,N,2,8,2,0
